In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# create directory for data
!mkdir data

# unzip files from zip folder
!unzip "/content/drive/MyDrive/PG/PB/Projekt badawczy - terminy medyczne/001_004.zip" -d "data/"

Archive:  /content/drive/MyDrive/PG/PB/Projekt badawczy - terminy medyczne/001_004.zip
   creating: data/001_004/
  inflating: data/001_004/001_004_online.csv  
  inflating: data/001_004/004_00005_20230711T124716.wav  
  inflating: data/001_004/004_00017_20230711T124847.wav  


In [5]:
data_directory = "./data/001_004/"

## Package installation

In [1]:
!pip install resemble-enhance --upgrade


## Imports

In [10]:
import pathlib
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from pydub import AudioSegment, effects
import os
import IPython.display as ipd

# RESEMBLE ENHANCE
import gradio as gr
import torch
import torchaudio

from resemble_enhance.enhancer.inference import denoise, enhance

In [7]:
data_path = pathlib.Path(data_directory)

files = list(data_path.glob('*.wav'))

In [6]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"


def _fn(path, solver, nfe, tau, denoising):
    if path is None:
        return None, None

    solver = solver.lower()
    nfe = int(nfe)
    lambd = 0.9 if denoising else 0.1

    dwav, sr = torchaudio.load(path)
    dwav = dwav.mean(dim=0)

    wav1, new_sr = denoise(dwav, sr, device)
    wav2, new_sr = enhance(dwav, sr, device, nfe=nfe, solver=solver, lambd=lambd, tau=tau)

    wav1 = wav1.cpu().numpy()
    wav2 = wav2.cpu().numpy()

    return (new_sr, wav1), (new_sr, wav2)


In [8]:
solver = "Midpoint"
nfe = 64
tau = 0.5
denoising = False

for file_path in files:
  v1, v2 = _fn(file_path, solver, nfe, tau, denoising)
  enhanced_1 = v1[0]
  break

100%|██████████| 1/1 [07:22<00:00, 442.03s/it]


In [15]:
original_file, sr = librosa.load(files[0], sr=None)

enhanced_file = v1[1]
enhanced_file_fq = v1[0]

display(ipd.Audio(data=original_file, rate=sr))
display(ipd.Audio(data=enhanced_file, rate=enhanced_file_fq))
# print(v1[1])